In [ ]:
seed = 42
proj_id = 'cmat-315920'
root_path  = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph']:
    %run -i {code_path+f+'.py'}

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

for level in ['tabblock', 'cntyvtd', 'bg', 'tract', 'cnty']:
    g = Gerry(abbr='TX', level=level,
              election_filters=(
                "office='President' and race='general'",
#                 "office='USSen' and race='general'",
#                 "office like 'USRep%' and race='general'",
              ),
              refresh_tbl=refresh_tbl, refresh_all=refresh_all
             )
    g.MCMC(3)
self = g